In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras import losses
from keras.models import model_from_json
import keras.backend.tensorflow_backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [2]:
#dir = './sequence/*'
dir = './latest_sequence/bfs/*'

In [3]:
# file read
all_data = []
sequence_length = []
name = []
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[-1].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [4]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [5]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [6]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 500
steps_per_epoch = len(x_train)

In [7]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    loss2 = losses.kld(y_true, y_pred) # categorical_crossentropy
    return loss1 * 0.7 + loss2 * 0.3

In [8]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss=losses.mean_squared_error, optimizer='adam')
#lstm_autoencoder_500 = lstm_autoencoder

W1004 17:02:47.855664 140183748400896 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1004 17:02:47.864664 140183748400896 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1004 17:02:47.867667 140183748400896 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1004 17:02:48.361598 140183748400896 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [9]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [10]:
model_path = 'models/weights' + '{epoch:02d}-{loss:.4f}.h5'
early_stopping_callback = EarlyStopping(monitor='loss', patience=200)
checkpoint_callback = ModelCheckpoint(model_path, monitor='loss', verbose = 1, save_best_only = True, save_weights_only = True, mode='min')#, period=5)

In [11]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=500, steps_per_epoch=steps_per_epoch, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback])

W1004 17:02:53.719572 140183748400896 deprecation.py:323] From /home/minji/anaconda3/envs/graph/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1004 17:02:55.021816 140183748400896 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1004 17:02:55.124544 140183748400896 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/350
2360/2360 [==============================] - 21s 9ms/step - loss: 38.3778

Epoch 00001: loss improved from inf to 38.37784, saving model to models/weights01-38.3778.h5
Epoch 2/350
2360/2360 [==============================] - 20s 8ms/step - loss: 37.4693

Epoch 00002: loss improved from 38.37784 to 37.46928, saving model to models/weights02-37.4693.h5
Epoch 3/350
2360/2360 [==============================] - 20s 8ms/step - loss: 20.1401

Epoch 00003: loss improved from 37.46928 to 20.14012, saving model to models/weights03-20.1401.h5
Epoch 4/350
2360/2360 [==============================] - 20s 9ms/step - loss: 13.4845

Epoch 00004: loss improved from 20.14012 to 13.48451, saving model to models/weights04-13.4845.h5
Epoch 5/350
2360/2360 [==============================] - 20s 8ms/step - loss: 11.7295

Epoch 00005: loss improved from 13.48451 to 11.72947, saving model to models/weights05-11.7295.h5
Epoch 6/350
2360/2360 [==============================] - 20s 9ms/step - loss: 11

2360/2360 [==============================] - 29s 12ms/step - loss: 1.6914

Epoch 00050: loss improved from 1.93430 to 1.69137, saving model to models/weights50-1.6914.h5
Epoch 51/350
2360/2360 [==============================] - 29s 12ms/step - loss: 1.9980

Epoch 00051: loss did not improve from 1.69137
Epoch 52/350
2360/2360 [==============================] - 29s 12ms/step - loss: 2.0341

Epoch 00052: loss did not improve from 1.69137
Epoch 53/350
2360/2360 [==============================] - 30s 13ms/step - loss: 1.6088

Epoch 00053: loss improved from 1.69137 to 1.60879, saving model to models/weights53-1.6088.h5
Epoch 54/350
2360/2360 [==============================] - 28s 12ms/step - loss: 1.5620

Epoch 00054: loss improved from 1.60879 to 1.56198, saving model to models/weights54-1.5620.h5
Epoch 55/350
2360/2360 [==============================] - 28s 12ms/step - loss: 1.8510

Epoch 00055: loss did not improve from 1.56198
Epoch 56/350
2360/2360 [==============================] - 2

2360/2360 [==============================] - 28s 12ms/step - loss: 0.5803

Epoch 00105: loss did not improve from 0.54588
Epoch 106/350
2360/2360 [==============================] - 28s 12ms/step - loss: 0.9095

Epoch 00106: loss did not improve from 0.54588
Epoch 107/350
2360/2360 [==============================] - 28s 12ms/step - loss: 0.7965

Epoch 00107: loss did not improve from 0.54588
Epoch 108/350
2360/2360 [==============================] - 29s 12ms/step - loss: 0.5807

Epoch 00108: loss did not improve from 0.54588
Epoch 109/350
2360/2360 [==============================] - 29s 12ms/step - loss: 0.7437

Epoch 00109: loss did not improve from 0.54588
Epoch 110/350
2360/2360 [==============================] - 31s 13ms/step - loss: 0.6975

Epoch 00110: loss did not improve from 0.54588
Epoch 111/350
2360/2360 [==============================] - 27s 12ms/step - loss: 0.6215

Epoch 00111: loss did not improve from 0.54588
Epoch 112/350
2360/2360 [==============================] - 28s

2360/2360 [==============================] - 28s 12ms/step - loss: 0.5173

Epoch 00162: loss did not improve from 0.22042
Epoch 163/350
2360/2360 [==============================] - 29s 12ms/step - loss: 0.3351

Epoch 00163: loss did not improve from 0.22042
Epoch 164/350
2360/2360 [==============================] - 28s 12ms/step - loss: 0.4883

Epoch 00164: loss did not improve from 0.22042
Epoch 165/350
2360/2360 [==============================] - 28s 12ms/step - loss: 0.3194

Epoch 00165: loss did not improve from 0.22042
Epoch 166/350
2360/2360 [==============================] - 28s 12ms/step - loss: 0.4231

Epoch 00166: loss did not improve from 0.22042
Epoch 167/350
2360/2360 [==============================] - 29s 12ms/step - loss: 0.3425

Epoch 00167: loss did not improve from 0.22042
Epoch 168/350
2360/2360 [==============================] - 28s 12ms/step - loss: 0.4977

Epoch 00168: loss did not improve from 0.22042
Epoch 169/350
2360/2360 [==============================] - 29s

2360/2360 [==============================] - 29s 12ms/step - loss: 0.1328

Epoch 00279: loss did not improve from 0.10716
Epoch 280/350
2360/2360 [==============================] - 29s 12ms/step - loss: 0.1929

Epoch 00280: loss did not improve from 0.10716
Epoch 281/350
2360/2360 [==============================] - 29s 12ms/step - loss: 0.2501

Epoch 00281: loss did not improve from 0.10716
Epoch 282/350
2360/2360 [==============================] - 29s 12ms/step - loss: 0.1900

Epoch 00282: loss did not improve from 0.10716
Epoch 283/350
2360/2360 [==============================] - 29s 12ms/step - loss: 0.1367

Epoch 00283: loss did not improve from 0.10716
Epoch 284/350
2360/2360 [==============================] - 29s 12ms/step - loss: 0.1179

Epoch 00284: loss did not improve from 0.10716
Epoch 285/350
2360/2360 [==============================] - 29s 12ms/step - loss: 0.1256

Epoch 00285: loss did not improve from 0.10716
Epoch 286/350
2360/2360 [==============================] - 29s

In [12]:
# MODEL SAVE
model_json = lstm_autoencoder.to_json()
filename = input('filename: ') #'latest_lstmae' #
with open('model/' + filename + '.json', 'w') as file:
    file.write(model_json)
lstm_autoencoder.save_weights('model/weights_' +  filename + '.h5')

In [15]:
# MODEL LOAD
loaded_model = model_from_json(open('model/' +filename + '.json').read())
loaded_model.load_weights('model/weights_' + filename + '.h5')

In [13]:
loaded_model = lstm_autoencoder

In [16]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

2.04192586293003


In [ ]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [ ]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])